# Graphs used in latex
## Graphs for second method

In [ ]:
%run ../../lib/basic-import.ipynb
%run ../../lib/plot-functions.ipynb
%run ../../lib/statistic-functions.ipynb

In [ ]:
import os.path
from bisect import *
import matplotlib.transforms as transforms
from collections import Counter
from scipy.stats import norm
from statistics import median, mode

In [ ]:
#Rounds up to 2 decimals, and calculate the percentage.
def percentage(part, whole):
  return round(100 * float(part)/float(whole),2)

In [ ]:
mean_dict= dist_average()
def plot_graphs_first():
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
    allDistances = {}
    countDistances = {}

    for participant in participant_files:
        ID = participant[:-4]
        add_df_338 = add_df[ID]
        add_df_338 = add_df_338[~add_df_338.isnull().all(1)]
        add_df_338 = add_df_338.asfreq('s')
        
        distances = list(add_df_338['distance'][add_df_338['lap'].diff() == 1])

        for i in range(len(distances)):
            if i >= 5:
                continue
            if i in allDistances:
                allDistances[i].append(distances[i])
            else:
                allDistances[i] = [distances[i]]

    #this shows distance the first plots.
    for i in range(len(allDistances)):
        fig, ax = plt.subplots(figsize=(10,4))

        distanceSort = allDistances[i]
        distanceSort.sort()

        df = pd.DataFrame(distanceSort, index=range(len(allDistances[i])))

        if i > 3:
            title = 'press {}'.format(i)
        else:
            title = 'lap {}'.format(i)

        df.plot(kind='bar',ax=ax, color='green',legend=False, title=title)
        ax.set_xticks([])

        if i <= 3:
            ax.axhline(y=mean_dict[i], color='red', linestyle='--', lw=0.7)
            ax.text(0,mean_dict[i], "{:.0f}".format(mean_dict[i]), color="red")

        ax.yaxis.label.set_color('green')
        ax.tick_params(axis='y', colors='green')

        fig.savefig('lap{}m1d.png'.format(i), format="png", bbox_inches='tight')

    #this shows number of times for one value
    #for i in range(len(allDistances)):
        #rounded to the nearest tens:
        #only used for the amount plots.
        #roundedDistances = [round(x,-1) for x in allDistances[i]] 
        #This counts:
        #countDistances[i] = dict(Counter(roundedDistances))

    for i in range(len(allDistances)):
        fig, ax = plt.subplots(figsize=(10,4))
        ax2 = ax.twinx()

        df = pd.DataFrame(allDistances[i])

        if i > 3:
            title = 'press {}'.format(i)
        else:
            title = 'lap {}'.format(i)

        df.plot.kde(ax=ax, color='blue',legend=False, title=title)
        df.plot.hist(ax=ax2, color='red',legend=False, alpha=0.1)
        ax.set_xlim(0,)

        #median
        pmedian = median(allDistances[i])
        ax.axvline(pmedian, color='purple', lw=1)
        #mean
        #pmean = mean(allDistances[i])
        #ax.axvline(pmean, color='red', lw=1)
        
        #1 standard deviation
        pstdev = stdev(allDistances[i])
        ax.axvline(pmedian + pstdev, color='blue', linestyle='--', lw=0.7)
        ax.axvline(pmedian - pstdev, color='blue', linestyle='--', lw=0.7)

        #2 standar deviation
        ax.axvline(pmedian + 2*pstdev, color='green', linestyle='--', lw=0.7)
        ax.axvline(pmedian - 2*pstdev, color='green', linestyle='--', lw=0.7)


        ax.yaxis.label.set_color('blue')
        ax.tick_params(axis='y', colors='blue')
        ax2.yaxis.label.set_color('red')
        ax2.tick_params(axis='y', colors='red')

        fig.savefig("lap{}m1c.png".format(i), format="png", bbox_inches='tight')

    for i in range(len(countDistances)):
        fig, ax = plt.subplots(figsize=(10,4))

        df = pd.Series(countDistances[i])
        df.index = map(str,df.index)
        if i > 3:
            title = 'lap >={}'.format(i)
        else:
            title = 'lap {}'.format(i)
        

        ax.set_ylabel('amount')
        ax.set_xlabel('Distance')
        df.plot(kind='bar',ax=ax, color='blue',legend=False, title=title, rot=45)
        
        plt.tick_params(axis='x', which='major', labelsize=7)


        fig.savefig("lap{}m1c.png".format(i), format="png", bbox_inches='tight')

    #add into latex pdf plots, method1
    if not os.path.exists('../latexPDF/First_method'):
        os.makedirs('../latexPDF/First_method')
    if not os.path.exists('../latexPDF/First_method/Values'):
        os.makedirs('../latexPDF/First_method/Values')
    if not os.path.exists('../latexPDF/First_method/Counter'):
        os.makedirs('../latexPDF/First_method/Counter')

    for i in range(len(allDistances)):
        if os.path.exists('../latexPDF/First_method/Counter/lap{}m1c.png'.format(i)):
            os.remove('../latexPDF/First_method/Counter/lap{}m1c.png'.format(i))
        os.rename('./lap{}m1c.png'.format(i), '../latexPDF/First_method/Counter/lap{}m1c.png'.format(i))

        if os.path.exists('../latexPDF/First_method/Values/lap{}m1d.png'.format(i)):
            os.remove('../latexPDF/First_method/Values/lap{}m1d.png'.format(i))
        os.rename('./lap{}m1d.png'.format(i), '../latexPDF/First_method/Values/lap{}m1d.png'.format(i))
        
plot_graphs_first()

In [ ]:
mean_dict = dist_average_percent()
def plot_graphs_second():
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
    allPercentages = {}
    countPercentages = {}

    for participant in participant_files:
        ID = participant[:-4]
        add_df_338 = add_df[ID]
        add_df_338 = add_df_338[~add_df_338.isnull().all(1)]
        add_df_338 = add_df_338.asfreq('s')
        
        distances = list(add_df_338['distance'][add_df_338['lap'].diff() == 1])

        lastDistance = add_df_338['distance'][add_df_338.index[-1]]

        for i in range(len(distances)):
            if i > 4:
                allPercentages[4].append(percentage(distances[i],lastDistance))
                continue
            if i in allPercentages:
                allPercentages[i].append(percentage(distances[i],lastDistance))
            else:
                allPercentages[i] = [percentage(distances[i],lastDistance)]

    #this shows percentage.
    for i in range(len(allPercentages)):
        fig, ax = plt.subplots(figsize=(10,4))

        percentageSort = allPercentages[i]
        percentageSort.sort()

        df = pd.DataFrame(percentageSort, index=range(len(allPercentages[i])))
        if i > 3:
            title = 'lap >={}'.format(i)
        else:
            title = 'lap {}'.format(i)
        df.plot(kind='bar',ax=ax, color='green',legend=False, title=title)
        
        if i <= 3:
            ax.axhline(y=mean_dict[i], color='r', linestyle='--', lw=0.7)
            ax.text(0,mean_dict[i], "{}".format(mean_dict[i]), color="red")

        ax.set_ylabel('percent of full distance')
        ax.set_xticks([])

        fig.savefig('lap{}m2d.png'.format(i), format="png", bbox_inches='tight')

    #this shows number of times for one value
    for i in range(len(allPercentages)):
        countPercentages[i] = dict(Counter(allPercentages[i]))

    for i in range(len(countPercentages)):
        fig, ax = plt.subplots(figsize=(10,4))

        df = pd.Series(countPercentages[i])
        df.index = map(str,df.index)

        if i > 3:
            title = 'lap >={}'.format(i)
        else:
            title = 'lap {}'.format(i)
        
        ax.set_ylabel('amount')
        ax.set_xlabel('percent of full distance')
        df.plot(kind='bar',ax=ax, color='blue',legend=False, title=title, rot=45)

        fig.savefig('lap{}m2c.png'.format(i), format="png", bbox_inches='tight')

    #add into latex pdf plots, method1
    if not os.path.exists('../latexPDF/Second_method'):
        os.makedirs('../latexPDF/Second_method')
    if not os.path.exists('../latexPDF/Second_method/Values'):
        os.makedirs('../latexPDF/Second_method/Values')
    if not os.path.exists('../latexPDF/Second_method/Counter'):
        os.makedirs('../latexPDF/Second_method/Counter')

    for i in range(len(allPercentages)):
        if os.path.exists('../latexPDF/Second_method/Counter/lap{}m2c.png'.format(i)):
            os.remove('../latexPDF/Second_method/Counter/lap{}m2c.png'.format(i))
        os.rename('./lap{}m2c.png'.format(i), '../latexPDF/Second_method/Counter/lap{}m2c.png'.format(i))

        if os.path.exists('../latexPDF/Second_method/Values/lap{}m2d.png'.format(i)):
            os.remove('../latexPDF/Second_method/Values/lap{}m2d.png'.format(i))
        os.rename('./lap{}m2d.png'.format(i), '../latexPDF/Second_method/Values/lap{}m2d.png'.format(i))

plot_graphs_second()